In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import pandas as pd
import numpy as np
import regex as re

In [5]:
from fastai.callback.fp16 import *

In [6]:
def get_y(r): return parent_label(r).split(" ")

In [13]:
learn_inf = load_learner(Path("export.pkl", cpu=False)).to_fp16()

In [8]:
path = Path("../data/test")

In [9]:
images = get_image_files(path)

In [10]:
images[0].name

'008FWT.JPG'

In [ ]:
df = pd.DataFrame(columns=['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat'])

for image in images:
    file_id = image.name.split(".")[0]
    _,_,pred = learn_inf.predict(image)
    new_row = { 'ID':file_id, 'leaf_rust':pred[2], 'stem_rust':pred[1], 'healthy_wheat':pred[0] }
    df.append(new_row, ignore_index=True)

In [ ]:
df.to_csv(path="answer.cs")